Published on October 15, 2025. By Marília Prata. mpwolke

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://i.ytimg.com/vi/ONiWugVEf2s/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLD0jFw0JEZbTyutoSeUBgU4L4zLBw)Youtube

## Competition Citation:

@misc{cafa-6-protein-function-prediction,

    author = {Iddo Friedberg and Predrag Radivojac and Paul D Thomas and An Phan and M. Clara De Paolis Kaluza and Damiano Piovesan and Parnal Joshi and Chris Mungall and Martyna Plomecka and Walter Reade and María Cruz},
    
    title = {CAFA 6 Protein Function Prediction},
    year = {2025},
    
    howpublished = {\url{https://kaggle.com/competitions/cafa-6-protein-function-prediction}},
    note = {Kaggle}
}

## Prospective data Competition 

"This is a prospective (i.e., future) data competition. Many proteins in the Test data do not currently have any assigned functions. Proteins having one or more of their functions published by researchers during the curation phase of the competition will comprise the future test set. Final leaderboard scores will be calculated after the curation phase of the competition."

"The organizers provide a set of protein sequences on which the participants are asked to predict **Gene Ontology (GO)** terms in each of the three subontologies: **Molecular Function (MF)**, **Biological Process (BP), and Cellular Component (CC)**. This set of sequences is referred to as the test superset."

https://www.kaggle.com/competitions/cafa-6-protein-function-prediction/overview

## GOATOOLS: A Python library for Gene Ontology analyses

In [ ]:
!pip install goatools

## OBO parser

In [ ]:
# Import the OBO parser from GOATools
from goatools import obo_parser

In [ ]:
!pip install wget

In [ ]:
import wget

## OBO path

In [ ]:
#By Alexander Chervov https://www.kaggle.com/code/alexandervc/gene-ontology-python-tutorial

obo_path = "../input/cafa-6-protein-function-prediction/Train/go-basic.obo"

go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = os.getcwd() + '/data'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.mkdir(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'

In [ ]:
!pip install Bio

## Fasta file

That's not fast. It simply stopped my Notebook AGAIN! I wrote that two years ago.

## **It seems I'm Homo inhabilis.**

In [ ]:
#https://stackoverflow.com/questions/29805642/learning-to-parse-a-fasta-file-with-python

def read_fasta(fp):
        name, seq = None, []
        for line in fp:
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line, []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))

with open('../input/cafa-6-protein-function-prediction/Train/train_sequences.fasta') as fp:
    for name, seq in read_fasta(fp):
        print(name, seq)

### I simply forgot what I done 2y ago 🥴 awkward :)

## Load IA tsv

"Information accretion: **IA.tsv** contains the information accretion (weights) for each GO term. These weights are used to compute weighted precision and recall, as described in the Evaluation section."

https://www.kaggle.com/competitions/cafa-6-protein-function-prediction/data

In [ ]:
IA = pd.read_csv('../input/cafa-6-protein-function-prediction/IA.tsv', sep='\t')
IA.head()

## Load Train_taxonomy

"Taxonomy: **train_taxonomy.tsv** contains the list of proteins and the species to which they belong, represented by a taxonomic identifier (taxon ID). The first column is the protein UniProt accession ID and the second is the taxon ID."

https://www.kaggle.com/competitions/cafa-6-protein-function-prediction/data

In [ ]:
train = pd.read_csv('../input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv', sep='\t')
train.tail()

### 9606 is Homo sapiens on testsuperset-taxon-list

In [ ]:
train['9606'].value_counts()

## Load Taxon-list

In [ ]:
lis = pd.read_csv('../input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv', sep='\t')
lis.head()

In [ ]:
lis['Species'].value_counts()

In [ ]:
#Code by Taha07  https://www.kaggle.com/taha07/data-scientists-jobs-analysis-visualization/notebook

from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'purple',
                      height =2000,
                      width = 2000
                     ).generate(str(lis["Species"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Taxonomy List")
plt.show()

### That Rattus is this close to the sapiens.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ1f2LX4-Auoyu5UllVObn0ChIAf4--GKGq8Q&s)

#### Fine, that's just a squirrel.

## Load train_terms

In [ ]:
ter = pd.read_csv('../input/cafa-6-protein-function-prediction/Train/train_terms.tsv', sep='\t' )
ter.head()

## Load sample_submission (on_bad_lines='skip')

In [ ]:
sub = pd.read_csv('../input/cafa-6-protein-function-prediction/sample_submission.tsv', sep='\t', on_bad_lines='skip')
sub.head()

In [ ]:
homo_sapiens = lis[(lis['Species']=='Homo sapiens')].reset_index(drop=True)
homo_sapiens.head()

## Draft Session: 1h:16m.

### Time to Homo inhabilis clock out.

#Acknowledgements:

Alexander Chervov https://www.kaggle.com/code/alexandervc/gene-ontology-python-tutorial

Marília Prata https://www.kaggle.com/code/mpwolke/cafa-5-protein-prediction/notebook

Marília Prata https://www.kaggle.com/code/mpwolke/danio-rerio/notebook